## T019 Molecular dynamics simulation
* https://projects.volkamerlab.org/teachopencadd/talktorials/T019_md_simulation.html

### TOC
#### 1. Preparation of complex pbd
* Download PDB file
* Prepare the protein ligand complex
    * Protein preparation
    * Ligand preparation
    * Merge protein and ligand

#### 2. Create system for MD simulation
* MD simulation set up
    * Force field
    * System - protein, ligand, water, ions
#### 3. Run MD simulation
* Perform the MD simulation
* Download results

### Some tips
* For simulations under periodic boundary conditions, it is recommended to use a simulation box large enough, so that the simulated macromolecule does not come into contact with neighboring images of itself.

### Advantage of MD simulation
* MD give valuable insights into the highly dynamic process of ligand binding to their target.
* ligands may induce conformational changes in the macromolecule that can best accommodate the small molecule.
    * binding sites that are not observed in static ligand-free structures, but can be discovered with MD simulations, are sometimes called cryptic binding sites.
    * The identification of such binding sites with MD simulation can kickstart new drug discovery campaigns.
* Later in the drug discovery process, MD simulations can also be used to estimate the quality of computationally identified small molecules before performing more costly and time-intensive in vitro tests. 


In [12]:
# Load libaries
import copy
from pathlib import Path
import sys

import requests
from IPython.display import display
import numpy as np
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import mdtraj as md
import pdbfixer
import nglview as nv
import openmm as mm
import openmm.app as app
from openmm import unit
from openff.toolkit.topology import Molecule, Topology
from openmmforcefields.generators import GAFFTemplateGenerator


In [2]:
# create data directory if not exists
data_dir = Path("./data/EGPR_tutorial")
Path(data_dir).mkdir(parents=True, exist_ok=True)
result_dir = Path("./results/EGPR_tutorial")
Path(result_dir).mkdir(parents=True, exist_ok=True)

In [3]:

pdbid = "3POZ" # EGFR kinase domain
ligand_name = "03P"
# check the complex structure
solvated_model_path = str(result_dir / f"{pdbid}_{ligand_name}_solvated.pdb")
solvated_view = nv.show_structure_file(solvated_model_path)
solvated_view.add_representation("cartoon", selection="protein")
solvated_view.add_representation("licorice", selection="water")
solvated_view.add_representation("ball+stick", selection="ligand")
solvated_view.add_representation("spacefill", selection="ion")
#solvated_view.add_representation("unitcell")
solvated_view.camera = "orthographic"
solvated_view

NGLWidget()

In [4]:
# Load the solvated structure using OpenMM
solvated_model = app.PDBFile(solvated_model_path)
model_topology = solvated_model.topology
model_positions = solvated_model.positions

In [5]:
print("Number of atoms in the system:", model_topology.getNumAtoms())

Number of atoms in the system: 52862


### System
* The chosen Langevin Integrator uses Langevin equations. 
* A list of all different kinds of integrators can be found in the [OpenMM Docs](http://docs.openmm.org/development/api-python/library.html#integrators). 
* For further insight into the Langevin Integrator, we recommend reading about Langevin equations, e.g. on [Wikipedia](https://en.wikipedia.org/wiki/Langevin_equation).

In [6]:
def generate_forcefield(
    rdkit_mol=None, protein_ff="amber14-all.xml", solvent_ff="amber14/tip3pfb.xml"
):
    """
    Generate an OpenMM Forcefield object and register a small molecule.

    Parameters
    ----------
    rdkit_mol: rdkit.Chem.rdchem.Mol
        Small molecule to register in the force field.
    protein_ff: string
        Name of the force field.
    solvent_ff: string
        Name of the solvent force field.

    Returns
    -------
    forcefield: openmm.app.Forcefield
        Forcefield with registered small molecule.
    """
    forcefield = app.ForceField(protein_ff, solvent_ff)

    if rdkit_mol is not None:
        gaff = GAFFTemplateGenerator(
            molecules=Molecule.from_rdkit(rdkit_mol, allow_undefined_stereo=True)
        )
        forcefield.registerTemplateGenerator(gaff.generator)

    return forcefield

In [7]:
rcsb_ligand = Chem.MolFromMolFile(str(result_dir / f"{ligand_name}_prepared.mol"))
forcefield = generate_forcefield(rcsb_ligand)

In [8]:
system = forcefield.createSystem(model_topology, nonbondedMethod=app.PME,
            nonbondedCutoff=1.0*unit.nanometers, #constraints=app.HBonds, rigidWater=True,
            ewaldErrorTolerance=0.0005)
# when creating the system, it uses all the cpu cores available.
integrator = mm.LangevinIntegrator(
    300 * unit.kelvin, 1.0 / unit.picoseconds, 2.0 * unit.femtoseconds
)
integrator.setConstraintTolerance(0.00001)
platform = mm.Platform.getPlatformByName("CUDA")
simulation = app.Simulation(model_topology, system, integrator, platform)
simulation.context.setPositions(model_positions)
# it takes several minutes to create the system and set the positions.

In [9]:
# minimize the energy
print('Minimizing...')
simulation.minimizeEnergy()

Minimizing...


In [10]:
with open(str(result_dir / f"{pdbid}_{ligand_name}_minimized.pdb"), "w") as f:
    app.PDBFile.writeFile(simulation.topology, 
                          simulation.context.getState(getPositions=True, enforcePeriodicBox=True).getPositions(), 
                          f, 
                          keepIds=True)
# for analysis, we need to save the minimized structure.

In [11]:
# equilibrate for 100 steps
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(100)

Equilibrating...


In [13]:
# append reporters
# you can save the trajectory with dcd or xtc format.
# xtc is compressed (GROMACS trajectory)
# dcd is not compressed (CHARMM trajectory), it consumes more disk space 3 times.
simulation.reporters.append(
    app.XTCReporter(str(result_dir / f"{pdbid}_{ligand_name}_trajectory.xtc"), 
                reportInterval=1000))
simulation.reporters.append(
    app.StateDataReporter(sys.stdout, 
                          reportInterval=1000,
                          step=True, 
                          potentialEnergy=True, 
                          temperature=True, 
                          progress=True, 
                          remainingTime=True, 
                          speed=True, 
                          totalSteps=250000, # 2 fs * 250000 = 500 ps
                          separator='\t'))

In [14]:
# run 0.5 ns of production simulation, it takes about 7 min
# for 50 ns, it takes about 12 hrs on a single GPU (RTX A2000 in HP server)
print('Running Production...')
simulation.step(250000)
print('Done!')

Running Production...
#"Progress (%)"	"Step"	"Potential Energy (kJ/mole)"	"Temperature (K)"	"Speed (ns/day)"	"Time Remaining"
0.4%	1000	-854574.8238048018	275.7676778713975	0	--
0.8%	2000	-840553.3120860518	294.1277057349834	102	6:58
1.2%	3000	-837691.3433360518	302.17443338328854	102	6:56
1.6%	4000	-835508.5210704268	299.9103313086578	102	6:57
2.0%	5000	-835426.4390391768	298.35740028117453	102	6:54
2.4%	6000	-836141.6909923018	300.22877765343645	102	6:52
2.8%	7000	-836574.3648204268	300.01345982929206	102	6:50
3.2%	8000	-837074.9175548018	299.8332918664327	102	6:49
3.6%	9000	-837403.7339610518	301.22163760140353	102	6:48
4.0%	10000	-836763.5952891768	300.536840504335	102	6:46
4.4%	11000	-836751.2769298018	300.9437017084883	102	6:44
4.8%	12000	-838105.8179454268	298.57620112134094	102	6:43
5.2%	13000	-836353.6616954268	300.5145986779408	102	6:41
5.6%	14000	-836101.7046641768	301.797852976507	102	6:40
6.0%	15000	-837371.4468516768	301.39738317177523	102	6:39
6.4%	16000	-838780.21247667